<a href="https://colab.research.google.com/github/LuanEverton/Project-CHATBOT-Amazon/blob/main/codigo_chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Código CHATBOT Amazon

In [ ]:
#!pip install openai twilio python-dotenv
#!pip install requests
#!pip install openai
#!pip install boto3



In [ ]:
import os #biblioteca os utilizada para acessar compornentes do sistema


#aqui armazenamos as credenciais com o os.environ para que possa ser acessada com o getenv() depois sem que as credenciais sejam expostas.
#nesse caso o os.environ é necessário por ser criado no colab
os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_OPENAI"
os.environ["TWILIO_SID"] = "SEU_TWILIO_SID"
os.environ["TWILIO_AUTH_TOKEN"] = "SEU_TWILIO_AUTH_TOKEN"
os.environ["TWILIO_WHATSAPP_NUMBER"] = "whatsapp:+14155238886"
os.environ["WHATSAPP_GROUP_NUMBER"] = "whatsapp:+SEU_NUMERO"


In [ ]:
#bibliotecas utilzadas no chatbot

import requests
import hashlib
import hmac
import time
import base64
import openai
import boto3

In [ ]:
#configurar as credeciais da API AMAZON

AWS_ACESS_KEY = os.getenv("AWS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")
ASSOCIATE_TAG = os.getenv("ASSOCIATE_TAG")
REGION = "region"
ENDPOINT = "websevice.amazon.com"



Função padrão para serviços que utlizam a API Service AWS, essa função cria sua assinatura digital que é obrigatória para utilização do serviço.



In [ ]:
def CriandoAssinatura(paramts,secret_key):
  sorted_params = "&".join(f"{k}={params[k]}" for k in sorted(params))
  string_to_sign = f"GET\n{ENDPOINT}\n/onca/xml\n{sorted_params}"
  signature = base64.b64encode(hmac.new(
       secret_key.encode("utf-8"),
        string_to_sign.encode("utf-8"),
        hashlib.sha256
    ).digest()).decode("utf-8")

Função padrão de busca.
Você pode fazer algumas alterações.


*  ***SerchIndex:*** você pode alterar para buscar categorias diferentes, por exemplo, "Eletronics" ou "Books".
* ***ResponseGroup***: Você pode alterar para trazer mais ou menos informações do
produto.
* Os dados são extraidos no formato **XML.**   






In [ ]:
def buscar_produtos_amazon(keyword):
    """
    Busca produtos na Amazon com base na palavra-chave fornecida.
    """
    params = {
        "Service": "AWSECommerceService",
        "Operation": "ItemSearch",
        "AWSAccessKeyId": AWS_ACCESS_KEY,
        "AssociateTag": ASSOCIATE_TAG,
        "SearchIndex": "Promotion",
        "Keywords": keyword,
        "ResponseGroup": "Images,ItemAttributes,Offers",
        "Timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())
    }

    params["Signature"] = criar_assinatura(params, AWS_SECRET_KEY)

    url = f"https://{ENDPOINT}/onca/xml"
    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.text  # Aqui podemos processar os dados para extrair informações úteis.
    else:
        return f"Erro: {response.status_code} - {response.text}"

Funções para mandar o prompt para o CHATGPT e retornar o mansagem promocional

In [ ]:
def GerarMensagemPromo(categoria,produto,link):
    prompt = f"Crie uma mensagem criativa de promoção na categoria {categoria}, do produto {produto} e que pode ser encontrado neste link: {link}."
    try:
        response = openai.Completion.create(
          model="gpt-3.5",
          prompt=prompt,
          max_tokens=150,
          temperature=0.9
        )

        mensagem = response.choices[0].text.strip()
        return mensagem

    except Exception as e:
      print("Houve {} erro ao gerar a mensagem".format((e)))
      return None


Função para enviar a mensagem para o grupo do WhatsApp

In [ ]:
def EnviarMensagemWhatsApp(mensagem):
  cliente = twilio.Client(os.getenv("TWILIO_SID"), os.getenv("TWILIO_AUTH_TOKEN"))
  mensagem = cliente.messages.create(
      corpo = mensagem,
      de = 'WhatsApp:' + os.getenv('TWILIO_WHATSAPP_NUMBER'),
      para = 'WhatsApp:' + os.getenv('WHATSAPP_GROUP_NUMBER')
  )

In [ ]:
client = boto3.client(
    's3'
    aws_access_key_id = AWS_ACESS_KEY,
    aws_secret_access_key = AWS_SECRET_KEY,
    region_name = REGION
)
